In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.34.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# Model from Hugging Face hub
base_model = "mistralai/Mistral-7B-v0.1"

# New instruction dataset
data_names="drive/MyDrive/game data template train.csv"
test_data_names = "drive/MyDrive/game data template validation.csv"
# Fine-tuned model
new_model = "zyxdream/Mistral-7B-chess"


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

## Output directory where the model predictions and checkpoints will be stored

# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# %ls

output_dir = "./Mistral-7B-chess-results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [4]:
testset = load_dataset('csv', data_files=[test_data_names],split='train') # actually the whole set is for validation
print(testset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 20
})


In [5]:
################################################################################
# Load base model with adopter
# First, we will reload the base mode and fine-tuned adapter using peft. The function below will attach the adapter with the base model.
################################################################################


# base_model_reload = AutoModelForCausalLM.from_pretrained(
#         base_model,
#         return_dict=True,
#         low_cpu_mem_usage=True,
#         #device_map="auto",
#         trust_remote_code=True,
#         offload_folder="offload",
#         offload_state_dict = True
# )

# Reload model in FP16 and merge it with LoRA weights
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto"
)


# from accelerate import disk_offload
# disk_offload(model=base_model_reload, offload_dir="offload")


# model = PeftModel.from_pretrained(base_model_reload, new_model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
testset = load_dataset('csv', data_files=[test_data_names],split='train') # actually the whole set is for validation
print(testset)

Dataset({
    features: ['text'],
    num_rows: 20
})


In [13]:
# Run text generation with fine-tuned model on validation set



for row in testset['text']:
  print("## row ##\n", row)
  inputs = row[:row.find('[/INST]')+7]
  print("## input ##\n", inputs)
  inputs = tokenizer.encode(inputs, return_tensors="pt").to("cuda")
  outputs_ids = model.generate(input_ids=inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=500, do_sample=True)
  outputs = tokenizer.batch_decode(outputs_ids)[0]
  print("Output: ***\n")
  print(outputs)
  print("***")

## row ##
 <s>[INST] You are given notations of chess board positions and the next chess move. Explain why it is the best move.     ### Chess board positions: r2q1rk1/pbp2p2/1p4p1/3p2N1/2P3nQ/3B4/PP3PPP/R3R1K1 b - - 0 17 ### Next Move: 17 …Nf6 ### Explanation: [/INST] And Black must dance to the tune, by bringing the knight back to f6.
Here too Black posts the knight on a good square, but not of his own free will. </s>
## input ##
 <s>[INST] You are given notations of chess board positions and the next chess move. Explain why it is the best move.     ### Chess board positions: r2q1rk1/pbp2p2/1p4p1/3p2N1/2P3nQ/3B4/PP3PPP/R3R1K1 b - - 0 17 ### Next Move: 17 …Nf6 ### Explanation: [/INST]
Output: ***

<s><s> [INST] You are given notations of chess board positions and the next chess move. Explain why it is the best move.     ### Chess board positions: r2q1rk1/pbp2p2/1p4p1/3p2N1/2P3nQ/3B4/PP3PPP/R3R1K1 b - - 0 17 ### Next Move: 17 …Nf6 ### Explanation: [/INST] The knight gets up and leaves t